# Geomapping the Hebrew Bible

In [1]:
import geopy.distance, math
from statistics import mean
import collections
import ipywidgets as widgets
from ipyleaflet import Marker, Map, Heatmap, MarkerCluster, Circle, basemaps, basemap_to_tiles

In [2]:
from tf.fabric import Fabric
from tf.extra.bhsa import Bhsa

In [3]:
locations = 'etcbc'
sources = ['bhsa', 'gps']
version = 'c'

In [4]:
modules = ['{}/{}/tf/{}'.format(locations, s, version) for s in sources]
TF = Fabric(modules=modules)

This is Text-Fabric 6.2.1
Api reference : https://dans-labs.github.io/text-fabric/Api/General/
Tutorial      : https://github.com/Dans-labs/text-fabric/blob/master/docs/tutorial.ipynb
Example data  : https://github.com/Dans-labs/text-fabric-data

116 features found and 0 ignored


In [5]:
api = TF.load('''
    otype
    book
    typ function
    sp vs
    gloss nametype lat long
''')
api.makeAvailableIn(globals())

  0.00s loading features ...
   |     0.09s B otype                from C:/Users/Ejer/text-fabric-data/etcbc/bhsa/tf/c
   |     0.02s B book                 from C:/Users/Ejer/text-fabric-data/etcbc/bhsa/tf/c
   |     0.23s B typ                  from C:/Users/Ejer/text-fabric-data/etcbc/bhsa/tf/c
   |     0.08s B function             from C:/Users/Ejer/text-fabric-data/etcbc/bhsa/tf/c
   |     0.14s B sp                   from C:/Users/Ejer/text-fabric-data/etcbc/bhsa/tf/c
   |     0.18s B vs                   from C:/Users/Ejer/text-fabric-data/etcbc/bhsa/tf/c
   |     0.03s B gloss                from C:/Users/Ejer/text-fabric-data/etcbc/bhsa/tf/c
   |     0.02s B nametype             from C:/Users/Ejer/text-fabric-data/etcbc/bhsa/tf/c
   |     0.02s B lat                  from C:/Users/Ejer/text-fabric-data/etcbc/gps/tf/c
   |     0.02s B long                 from C:/Users/Ejer/text-fabric-data/etcbc/gps/tf/c
  6.41s All features loaded/computed - for details use loadLog()


[('computed-data', ('C Computed', 'Call AllComputeds', 'Cs ComputedString')),
 ('edge-features', ('E Edge', 'Eall AllEdges', 'Es EdgeString')),
 ('loading', ('TF', 'ensureLoaded', 'ignored', 'loadLog')),
 ('locality', ('L Locality',)),
 ('messaging', ('cache', 'error', 'indent', 'info', 'reset')),
 ('navigating-nodes', ('N Nodes', 'sortKey', 'otypeRank', 'sortNodes')),
 ('node-features', ('F Feature', 'Fall AllFeatures', 'Fs FeatureString')),
 ('searching', ('S Search',)),
 ('text', ('T Text',))]

## The `lat` and `long` features

In [6]:
count = []

for n in F.otype.s('lex'):
    if F.lat.v(n) != None:
        count.append(F.gloss.v(n))
        
print(f'Glosses with GPS coordinates: {len(count)}')

Glosses with GPS coordinates: 354


In [7]:
count[:10]

['Cush',
 'Tigris',
 'Asshur',
 'Euphrates',
 'Ararat',
 'Elishah',
 'Tarshish',
 'Egypt',
 'Sheba',
 'Dedan']

In [10]:
place = 'Jerusalem'

F.gloss.s(place)

(1441117, 1445589)

In [12]:
gloss = F.gloss.s(place)[1]

print(F.lat.v(gloss), F.long.v(gloss))

31.777444 35.234935


In [13]:
node = 1440694

print(f'GPS coordinates for {F.gloss.v(node)}: {F.lat.v(node)}, {F.long.v(node)}')

GPS coordinates for Nobah: None, None


### Creating Google Maps links

In [14]:
from IPython.core.display import display, HTML

In [15]:
def gmapsLink(node):
    link = 'http://www.google.com/maps/place/'
    lat = F.lat.v(node)
    long = F.long.v(node)
    link = f'<a href="{link}{lat},{long}" target="blank">{F.gloss.v(node)}</a>'
    
    return display(HTML(link))

In [16]:
gmapsLink(1437723)

In [17]:
gmapsLink(F.gloss.s('Azmon')[0])

In [16]:
for n in F.otype.s('lex'):
    if F.lat.v(n) != None:
        gmapsLink(n)

## Plotting locations to Google Maps widget

In [18]:
def marker_locations(gloss_list):
    gps_list = []
    for gloss in gloss_list:
        node = F.gloss.s(gloss)
        for n in node:
            if F.lat.v(n) != None: #Some glosses, e.g. Nobah, is both a person and a place. We assure that they have latitude
                gps = float(F.lat.v(n)), float(F.long.v(n))
                marker = Marker(location=gps, title=gloss, rise_on_hover = True, draggable = False)
                gps_list.append(marker)
                break #The loop is finished after the first postive result to avoid redundant place names.
    return gps_list

def getCenter(gloss_list):
    lat_list = []
    long_list = []
    for gloss in gloss_list:
        node = F.gloss.s(gloss)
        for n in node:
            if F.lat.v(n) != None:
                lat_list.append(float(F.lat.v(n)))
                long_list.append(float(F.long.v(n)))
                break
    center=(mean(lat_list),mean(long_list))
    return center

def createMap(place_list):  
    m = Map(center=getCenter(place_list), zoom=5)

    marker_cluster = MarkerCluster(
        markers=(marker_locations(place_list))
    )
    
    layer = basemap_to_tiles(basemaps.Esri.WorldTopoMap)
    m.add_layer(layer)

    m.add_layer(marker_cluster);
    return m

In [21]:
place = ['Shittim']

createMap(place)

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …

A dictionary of layers:

In [19]:
#basemaps

In [20]:
def allPlaces():
    all_places = []
    for n in F.otype.s('lex'):
        if F.lat.v(n) != None:
            all_places.append(F.gloss.v(n))
            
    return all_places

In [21]:
createMap(allPlaces())

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …

## Calculating distance between locations

The distance between two sets of coordinates is calculated with [Vincenty's formulae](https://en.wikipedia.org/wiki/Vincenty%27s_formulae) which is implemented in the Python module [geopy](https://pypi.org/project/geopy/). Vincenty's formulae calculate the distance between two points on the surface of a spheroid.

In [22]:
def calcDistance(place_tuple):
    gps_list = []
    for pl in place_tuple:
        node = F.gloss.s(pl)
        for n in node:
            if F.lat.v(n) != None:
                gps = float(F.lat.v(n)), float(F.long.v(n))
                gps_list.append(gps)
                break

    distance = geopy.distance.vincenty(gps_list[0], gps_list[1]).km
    return distance

In [24]:
place = ('Shiloh', 'Jerusalem')

dist = calcDistance(place)
print(f'{round(dist, 2)} km')

31.28 km


## Showing all locations within a radius

In [25]:
def createRadiusMap(center, km=10):
    place_list=allPlaces()
    places_within_radius = []
    for pl in place_list:
        comp_place = (center, pl)
        distance = calcDistance(comp_place)
        if distance <= km:
            places_within_radius.append(pl)
    
    marker_cluster = MarkerCluster(
        markers=marker_locations(places_within_radius)
    )
    
    zoom = 15*km**-0.146 # Zoom is calculated as a power function dependent on the radius.
    zoom = int(round(zoom))
    
    center_marker = marker_locations(center.split())
    m = Map(center=getCenter(center.split()), zoom=zoom)
    
    m.add_layer(marker_cluster);
    
    tile_layer = basemap_to_tiles(basemaps.Esri.WorldTopoMap)
    m.add_layer(tile_layer)
    
    circle = Circle()
    circle.location = getCenter(center.split())
    circle.radius = km*1000
    circle.fill = True
    circle.fill_color = "black"
    circle.fill_opacity = 0.04
    circle.color = "black"
    circle.weight = 1
    circle.opacity = .5
    
    m.add_layer(circle)
    
    display(m)

In [26]:
w = widgets.interact_manual(createRadiusMap, center="Jerusalem", km=(1,100))

interactive(children=(Text(value='Jerusalem', description='center'), IntSlider(value=10, description='km', min…

## Showing locations according to Biblical book

In [27]:
def bookRef():
    
    book_ref_dict = collections.defaultdict(lambda: collections.defaultdict(int))
    for n in F.otype.s('word'):
        lex = L.u(n, 'lex')[0]
        if F.lat.v(lex) != None:
            book = T.bookName(n)
            gloss = F.gloss.v(lex)
            book_ref_dict[book][F.gloss.v(lex)] += 1
        
    return book_ref_dict

In [28]:
book_ref_dict = bookRef()

In [29]:
def bookList(number):
    book_list = []

    for n in F.otype.s('book'):
        book_list.append(T.bookName(n))
    
    return book_list[number]

def createBookMap(number=1):
    book = bookList(number-1)
    display(book)
    
    global book_ref_dict
    try:
        book_ref_dict
    except:
        book_ref_dict = bookRef()
    place_list = book_ref_dict[book].keys()
    
    display(createMap(place_list))

In [30]:
w = widgets.interact(createBookMap, number=(1,39))

interactive(children=(IntSlider(value=1, description='number', max=39, min=1), Output()), _dom_classes=('widge…

In [31]:
def createBookHeatMap(number=1):
    book = bookList(number-1)
    
    display(book)
    
    global book_ref_dict
    try:
        book_ref_dict
    except:
        book_ref_dict = bookRef()

    book_ref = book_ref_dict[book]
    
    locations = []
    for pl in book_ref:
        node = F.gloss.s(pl)
        for n in node:
            if F.lat.v(n) != None:
                locations.append([float(F.lat.v(n)), float(F.long.v(n)), book_ref[pl]])
                break
    
    m = Map(center=getCenter(book_ref.keys()), zoom=6)
    
    heatmap = Heatmap(
        locations=locations,
        radius=20,
        min_opacity = 0.20,
        gradient = {0.2: 'blue', 0.4: 'cyan', 0.6: 'lime', 0.8: 'yellow', 1.0: 'red'}
    )
    
    tile_layer = basemap_to_tiles(basemaps.Esri.WorldTopoMap)
    m.add_layer(tile_layer)
            
    m.add_layer(heatmap);

    display(m)

In [32]:
w = widgets.interact(createBookHeatMap, number=(1,39), continuous_update=False)

interactive(children=(IntSlider(value=1, description='number', max=39, min=1), Output()), _dom_classes=('widge…